## Final Exam

In [1]:
# housekeeping - curiously, run this once, reset, then run again to remove errors
#!pip install -r requirements.txt
#
# Go to the last cell, and run it to load requirements.txt
# As an alternative, uncomment the above pip ine and execute
# this cell

## Setup Credentials for Google Colab

In [1]:
import os
from getpass import getpass
from google.colab import userdata
import warnings
warnings.filterwarnings('ignore')

# === CREDENTIAL SETUP WITH VALIDATION ===
def setup_credentials():
    """Setup and validate IBM Watson credentials"""
    try:
        # Try to get from userdata first
        ibm_api_key = userdata.get('IBM_API_KEY')
        ibm_project_id = userdata.get('IBM_PROJECT_ID')
        watsonx_apikey = userdata.get('WATSONX_APIKEY')

        # If not found, prompt user
        if not ibm_api_key or not ibm_project_id:
            print("⚠️  Credentials not found in userdata. Please enter manually:")
            ibm_api_key = getpass('IBM API Key: ')
            ibm_project_id = getpass('IBM Project ID: ')
            watsonx_apikey = ibm_api_key  # Often the same

        # Validate credentials format
        if not ibm_api_key or len(ibm_api_key) < 40:
            raise ValueError("IBM API Key appears invalid (too short)")
        if not ibm_project_id or len(ibm_project_id) < 30:
            raise ValueError("IBM Project ID appears invalid (too short)")

        # Set environment variables
        os.environ['IBM_API_KEY'] = ibm_api_key
        os.environ['IBM_PROJECT_ID'] = ibm_project_id
        os.environ['WATSONX_APIKEY'] = watsonx_apikey

        print("✅ Credentials configured successfully")
        return ibm_api_key, ibm_project_id, watsonx_apikey

    except Exception as e:
        print(f"❌ Error setting up credentials: {e}")
        raise

# === TEST CREDENTIALS FUNCTION ===
def test_credentials():
    """Test if credentials work by making a simple API call"""
    try:
        from ibm_watsonx_ai.foundation_models import ModelInference
        from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

        # Try to create a simple model instance
        model = ModelInference(
            model_id='mistralai/mixtral-8x7b-instruct-v01',
            params={GenParams.MAX_NEW_TOKENS: 10},
            credentials={
                "url": "https://us-south.ml.cloud.ibm.com",
                "apikey": os.environ['IBM_API_KEY']
            },
            project_id=os.environ['IBM_PROJECT_ID']
        )

        # Try a simple generation
        response = model.generate_text("Hello")
        print("✅ Credentials test successful!")
        return True

    except Exception as e:
        print(f"❌ Credentials test failed: {e}")
        return False


# Import Libraries

In [2]:

from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from ibm_watsonx_ai import Credentials
from langchain_ibm import WatsonxLLM, WatsonxEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
import gradio as gr
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')


# Initialize the LLM

In [3]:
## LLM

def get_llm():
    """Initialize the LLM with error handling"""
    try:
        model_id = 'mistralai/mixtral-8x7b-instruct-v01'
        parameters = {
            GenParams.MAX_NEW_TOKENS: 256,
            GenParams.TEMPERATURE: 0.5,
        }

        watsonx_llm = WatsonxLLM(
            model_id=model_id,
            url="https://us-south.ml.cloud.ibm.com",
            project_id=os.environ['IBM_PROJECT_ID'],
            apikey=os.environ['IBM_API_KEY'],
            params=parameters,
        )
        return watsonx_llm
    except Exception as e:
        print(f"❌ Error initializing LLM: {e}")
        raise


# Define the embedding model

In [4]:

## Embedding model

def watsonx_embedding():
    """Initialize embeddings with error handling"""
    try:
        embed_params = {
            EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
            EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
        }

        watsonx_embedding = WatsonxEmbeddings(
            model_id="ibm/slate-125m-english-rtrvr",
            url="https://us-south.ml.cloud.ibm.com",
            project_id=os.environ['IBM_PROJECT_ID'],
            apikey=os.environ['IBM_API_KEY'],
            params=embed_params,
        )
        return watsonx_embedding
    except Exception as e:
        print(f"❌ Error initializing embeddings: {e}")
        raise


# Define the PDF document loader

In [5]:

## Document loader

def document_loader(file):
    """Load PDF document"""
    try:
        loader = PyPDFLoader(file.name)
        loaded_document = loader.load()
        print(f"✅ Loaded {len(loaded_document)} pages from PDF")
        return loaded_document
    except Exception as e:
        print(f"❌ Error loading document: {e}")
        raise


# Define the text splitter

In [6]:

## Text splitter

def text_splitter(data):
    """Split text into chunks"""
    try:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=50,
            length_function=len,
        )
        chunks = text_splitter.split_documents(data)
        print(f"✅ Split document into {len(chunks)} chunks")
        return chunks
    except Exception as e:
        print(f"❌ Error splitting text: {e}")
        raise


# Define the vector store

In [7]:

## Vector db

def vector_database(chunks):
    """Create vector database"""
    try:
        embedding_model = watsonx_embedding()
        vectordb = Chroma.from_documents(chunks, embedding_model)
        print("✅ Created vector database")
        return vectordb
    except Exception as e:
        print(f"❌ Error creating vector database: {e}")
        raise


# Define the retriever

In [8]:

## Retriever

def retriever(file):
    """Create retriever from file"""
    try:
        splits = document_loader(file)
        chunks = text_splitter(splits)
        vectordb = vector_database(chunks)
        retriever = vectordb.as_retriever()
        print("✅ Created retriever")
        return retriever
    except Exception as e:
        print(f"❌ Error creating retriever: {e}")
        raise


# Define a question-answering chain

In [9]:

## QA Chain

def retriever_qa(file, query):
    """Main QA function with comprehensive error handling"""
    try:
        print(f"🔍 Processing query: {query}")
        print(f"📄 File: {file}")

        # Initialize components
        llm = get_llm()
        retriever_obj = retriever(file)

        # Create QA chain
        qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever_obj,
            return_source_documents=False
        )

        # Get response
        response = qa(query)
        print("✅ Generated response successfully")
        return response['result']

    except Exception as e:
        error_msg = f"❌ Error processing query: {str(e)}"
        print(error_msg)
        return f"Sorry, I encountered an error: {str(e)}\n\nPlease check your credentials and try again."


# Initialize Application

#Set up the Gradio interface

In [10]:
def main():
    """Main setup function"""
    print("🚀 Setting up RAG Application...")

    # Setup credentials
    setup_credentials()

    if False:
      # Test credentials
      if not test_credentials():
        print("❌ Credential test failed. Please check your IBM Watson credentials.")
        return None

    # Create Gradio interface
    rag_application = gr.Interface(
        fn=retriever_qa,
        allow_flagging="never",
        inputs=[
            gr.File(label="Upload PDF File", file_count="single", file_types=['.pdf'], type="filepath"),
            gr.Textbox(label="Input Query", lines=2, placeholder="Type your question here...")
        ],
        outputs=gr.Textbox(label="Output"),
        title="RAG Chatbot",
        description="Upload a PDF document and ask any question. The chatbot will try to answer using the provided document."
    )

    print("✅ RAG Application ready!")
    return rag_application


# Add code to launch the application

In [11]:

# Launch the app

# === LAUNCH ===
if __name__ == "__main__":
    # Setup and launch
    app = main()
    if app:
        app.launch(server_name="0.0.0.0", server_port=7860, debug=True)
    else:
        print("❌ Failed to initialize application")


🚀 Setting up RAG Application...
✅ Credentials configured successfully
✅ RAG Application ready!
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4cf3668a9deac05044.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


🔍 Processing query: Who is Lucas?

📄 File: /tmp/gradio/10f0c7e2f2baa100d9272013d04b917b3e72e6bf5e377e4e208d0c26719ba57c/lucas-beeler.pdf
✅ Loaded 2 pages from PDF
✅ Split document into 9 chunks
✅ Created vector database
✅ Created retriever
✅ Generated response successfully
🔍 Processing query: What are his strengths?

📄 File: /tmp/gradio/10f0c7e2f2baa100d9272013d04b917b3e72e6bf5e377e4e208d0c26719ba57c/lucas-beeler.pdf
✅ Loaded 2 pages from PDF
✅ Split document into 9 chunks
✅ Created vector database
✅ Created retriever
✅ Generated response successfully
🔍 Processing query: What are his weaknesses?


📄 File: /tmp/gradio/10f0c7e2f2baa100d9272013d04b917b3e72e6bf5e377e4e208d0c26719ba57c/lucas-beeler.pdf
✅ Loaded 2 pages from PDF
✅ Split document into 9 chunks
✅ Created vector database
✅ Created retriever
✅ Generated response successfully
🔍 Processing query: Is Lucas a strong candidate?


📄 File: /tmp/gradio/10f0c7e2f2baa100d9272013d04b917b3e72e6bf5e377e4e208d0c26719ba57c/lucas-beeler.pdf
✅ L

# Run this once, reset, then run again, then reset to install pip libraries
# You can also load the file into requirements.txt and use !pip install -r requirements.txt

In [ ]:
#
# This file is autogenerated by pip-compile with Python 3.12
# by the following command:
#
#    pip-compile
#
aiofiles==23.2.1
    # via gradio
aiohappyeyeballs==2.6.1
    # via aiohttp
aiohttp==3.12.13
    # via
    #   langchain
    #   langchain-community
aiosignal==1.3.2
    # via aiohttp
annotated-types==0.7.0
    # via pydantic
anyio==4.9.0
    # via
    #   gradio
    #   httpx
    #   starlette
    #   watchfiles
asgiref==3.8.1
    # via opentelemetry-instrumentation-asgi
attrs==25.3.0
    # via aiohttp
backoff==2.2.1
    # via posthog
bcrypt==4.3.0
    # via chromadb
build==1.2.2.post1
    # via chromadb
cachetools==5.5.2
    # via google-auth
certifi==2025.6.15
    # via
    #   httpcore
    #   httpx
    #   ibm-watsonx-ai
    #   kubernetes
    #   pulsar-client
    #   requests
charset-normalizer==3.4.2
    # via requests
chroma-hnswlib==0.7.3
    # via chromadb
chromadb==0.4.24
    # via -r requirements.in
click==8.2.1
    # via
    #   typer
    #   uvicorn
coloredlogs==15.0.1
    # via onnxruntime
contourpy==1.3.2
    # via matplotlib
cycler==0.12.1
    # via matplotlib
dataclasses-json==0.6.7
    # via langchain-community
distro==1.9.0
    # via posthog
durationpy==0.10
    # via kubernetes
fastapi==0.115.13
    # via
    #   chromadb
    #   gradio
ffmpy==0.6.0
    # via gradio
filelock==3.18.0
    # via huggingface-hub
flatbuffers==25.2.10
    # via onnxruntime
fonttools==4.58.4
    # via matplotlib
frozenlist==1.7.0
    # via
    #   aiohttp
    #   aiosignal
fsspec==2025.5.1
    # via
    #   -r requirements.in
    #   gradio-client
    #   huggingface-hub
google-auth==2.40.3
    # via
    #   -r requirements.in
    #   kubernetes
googleapis-common-protos==1.70.0
    # via opentelemetry-exporter-otlp-proto-grpc
gradio==4.44.0
    # via -r requirements.in
gradio-client==1.3.0
    # via gradio
greenlet==3.2.3
    # via sqlalchemy
grpcio==1.73.0
    # via
    #   chromadb
    #   opentelemetry-exporter-otlp-proto-grpc
h11==0.16.0
    # via
    #   httpcore
    #   uvicorn
hf-xet==1.1.5
    # via huggingface-hub
httpcore==1.0.9
    # via httpx
httptools==0.6.4
    # via uvicorn
httpx==0.28.1
    # via
    #   gradio
    #   gradio-client
    #   langsmith
huggingface-hub==0.33.0
    # via
    #   gradio
    #   gradio-client
    #   tokenizers
humanfriendly==10.0
    # via coloredlogs
ibm-cos-sdk==2.13.6
    # via ibm-watsonx-ai
ibm-cos-sdk-core==2.13.6
    # via
    #   ibm-cos-sdk
    #   ibm-cos-sdk-s3transfer
ibm-cos-sdk-s3transfer==2.13.6
    # via ibm-cos-sdk
ibm-watsonx-ai==1.1.2
    # via
    #   -r requirements.in
    #   langchain-ibm
idna==3.10
    # via
    #   anyio
    #   httpx
    #   requests
    #   yarl
importlib-metadata==8.7.0
    # via
    #   ibm-watsonx-ai
    #   opentelemetry-api
importlib-resources==6.5.2
    # via
    #   chromadb
    #   gradio
jedi==0.19.2
    # via -r requirements.in
jinja2==3.1.6
    # via gradio
jmespath==1.0.1
    # via
    #   ibm-cos-sdk
    #   ibm-cos-sdk-core
jsonpatch==1.33
    # via langchain-core
jsonpointer==3.0.0
    # via jsonpatch
kiwisolver==1.4.8
    # via matplotlib
kubernetes==33.1.0
    # via chromadb
langchain==0.2.11
    # via
    #   -r requirements.in
    #   langchain-community
langchain-community==0.2.10
    # via -r requirements.in
langchain-core==0.2.43
    # via
    #   langchain
    #   langchain-community
    #   langchain-ibm
    #   langchain-text-splitters
langchain-ibm==0.1.11
    # via -r requirements.in
langchain-text-splitters==0.2.4
    # via langchain
langsmith==0.1.147
    # via
    #   langchain
    #   langchain-community
    #   langchain-core
lomond==0.3.3
    # via ibm-watsonx-ai
markdown-it-py==3.0.0
    # via rich
markupsafe==2.1.5
    # via
    #   gradio
    #   jinja2
marshmallow==3.26.1
    # via dataclasses-json
matplotlib==3.10.3
    # via gradio
mdurl==0.1.2
    # via markdown-it-py
mmh3==5.1.0
    # via chromadb
mpmath==1.3.0
    # via sympy
multidict==6.5.0
    # via
    #   aiohttp
    #   yarl
mypy-extensions==1.1.0
    # via typing-inspect
numpy==1.26.4
    # via
    #   -r requirements.in
    #   chroma-hnswlib
    #   chromadb
    #   contourpy
    #   gradio
    #   langchain
    #   langchain-community
    #   matplotlib
    #   onnxruntime
    #   pandas
oauthlib==3.3.1
    # via
    #   kubernetes
    #   requests-oauthlib
onnxruntime==1.22.0
    # via chromadb
opentelemetry-api==1.34.1
    # via
    #   chromadb
    #   opentelemetry-exporter-otlp-proto-grpc
    #   opentelemetry-instrumentation
    #   opentelemetry-instrumentation-asgi
    #   opentelemetry-instrumentation-fastapi
    #   opentelemetry-sdk
    #   opentelemetry-semantic-conventions
opentelemetry-exporter-otlp-proto-common==1.34.1
    # via opentelemetry-exporter-otlp-proto-grpc
opentelemetry-exporter-otlp-proto-grpc==1.34.1
    # via chromadb
opentelemetry-instrumentation==0.55b1
    # via
    #   opentelemetry-instrumentation-asgi
    #   opentelemetry-instrumentation-fastapi
opentelemetry-instrumentation-asgi==0.55b1
    # via opentelemetry-instrumentation-fastapi
opentelemetry-instrumentation-fastapi==0.55b1
    # via chromadb
opentelemetry-proto==1.34.1
    # via
    #   opentelemetry-exporter-otlp-proto-common
    #   opentelemetry-exporter-otlp-proto-grpc
opentelemetry-sdk==1.34.1
    # via
    #   chromadb
    #   opentelemetry-exporter-otlp-proto-grpc
opentelemetry-semantic-conventions==0.55b1
    # via
    #   opentelemetry-instrumentation
    #   opentelemetry-instrumentation-asgi
    #   opentelemetry-instrumentation-fastapi
    #   opentelemetry-sdk
opentelemetry-util-http==0.55b1
    # via
    #   opentelemetry-instrumentation-asgi
    #   opentelemetry-instrumentation-fastapi
orjson==3.10.18
    # via
    #   chromadb
    #   gradio
    #   langsmith
overrides==7.7.0
    # via chromadb
packaging==24.2
    # via
    #   build
    #   gradio
    #   gradio-client
    #   huggingface-hub
    #   ibm-watsonx-ai
    #   langchain-core
    #   marshmallow
    #   matplotlib
    #   onnxruntime
    #   opentelemetry-instrumentation
pandas==2.1.4
    # via
    #   -r requirements.in
    #   gradio
    #   ibm-watsonx-ai
parso==0.8.4
    # via jedi
pillow==10.4.0
    # via
    #   gradio
    #   matplotlib
posthog==5.4.0
    # via chromadb
propcache==0.3.2
    # via
    #   aiohttp
    #   yarl
protobuf==5.29.5
    # via
    #   googleapis-common-protos
    #   onnxruntime
    #   opentelemetry-proto
pulsar-client==3.7.0
    # via chromadb
pyasn1==0.6.1
    # via
    #   pyasn1-modules
    #   rsa
pyasn1-modules==0.4.2
    # via google-auth
pydantic==2.9.1
    # via
    #   -r requirements.in
    #   chromadb
    #   fastapi
    #   gradio
    #   langchain
    #   langchain-core
    #   langsmith
pydantic-core==2.23.3
    # via pydantic
pydub==0.25.1
    # via gradio
pygments==2.19.2
    # via rich
pyparsing==3.2.3
    # via matplotlib
pypdf==4.3.1
    # via -r requirements.in
pypika==0.48.9
    # via chromadb
pyproject-hooks==1.2.0
    # via build
python-dateutil==2.9.0.post0
    # via
    #   ibm-cos-sdk-core
    #   kubernetes
    #   matplotlib
    #   pandas
    #   posthog
python-dotenv==1.1.0
    # via uvicorn
python-multipart==0.0.20
    # via gradio
pytz==2025.2
    # via pandas
pyyaml==6.0.2
    # via
    #   chromadb
    #   gradio
    #   huggingface-hub
    #   kubernetes
    #   langchain
    #   langchain-community
    #   langchain-core
    #   uvicorn
requests==2.32.2
    # via
    #   -r requirements.in
    #   chromadb
    #   huggingface-hub
    #   ibm-cos-sdk-core
    #   ibm-watsonx-ai
    #   kubernetes
    #   langchain
    #   langchain-community
    #   langsmith
    #   posthog
    #   requests-oauthlib
    #   requests-toolbelt
requests-oauthlib==2.0.0
    # via kubernetes
requests-toolbelt==1.0.0
    # via langsmith
rich==14.0.0
    # via
    #   -r requirements.in
    #   typer
rsa==4.9.1
    # via google-auth
ruff==0.12.0
    # via gradio
semantic-version==2.10.0
    # via gradio
shellingham==1.5.4
    # via typer
six==1.17.0
    # via
    #   kubernetes
    #   lomond
    #   posthog
    #   python-dateutil
sniffio==1.3.1
    # via anyio
sqlalchemy==2.0.41
    # via
    #   langchain
    #   langchain-community
starlette==0.46.2
    # via fastapi
sympy==1.14.0
    # via
    #   -r requirements.in
    #   onnxruntime
tabulate==0.9.0
    # via ibm-watsonx-ai
tenacity==8.5.0
    # via
    #   chromadb
    #   langchain
    #   langchain-community
    #   langchain-core
tokenizers==0.21.1
    # via chromadb
tomlkit==0.12.0
    # via gradio
tqdm==4.67.1
    # via
    #   chromadb
    #   huggingface-hub
typer==0.16.0
    # via
    #   chromadb
    #   gradio
typing-extensions==4.14.0
    # via
    #   anyio
    #   chromadb
    #   fastapi
    #   gradio
    #   gradio-client
    #   huggingface-hub
    #   langchain-core
    #   opentelemetry-api
    #   opentelemetry-exporter-otlp-proto-grpc
    #   opentelemetry-sdk
    #   opentelemetry-semantic-conventions
    #   pydantic
    #   pydantic-core
    #   sqlalchemy
    #   typer
    #   typing-inspect
typing-inspect==0.9.0
    # via dataclasses-json
tzdata==2025.2
    # via pandas
urllib3==2.5.0
    # via
    #   gradio
    #   ibm-cos-sdk-core
    #   ibm-watsonx-ai
    #   kubernetes
    #   requests
uvicorn[standard]==0.34.3
    # via
    #   chromadb
    #   gradio
uvloop==0.21.0
    # via uvicorn
watchfiles==1.1.0
    # via uvicorn
websocket-client==1.8.0
    # via kubernetes
websockets==12.0
    # via
    #   -r requirements.in
    #   gradio-client
    #   uvicorn
wrapt==1.17.2
    # via opentelemetry-instrumentation
yarl==1.20.1
    # via aiohttp
zipp==3.23.0
    # via importlib-metadata
